In [13]:
import pandas as pd
import numpy as np
import lightgbm as lgb

In [14]:
data_path = 'D:\\jupyter files\\data_waiting_time_project\\preprocess_data\\'

In [22]:
data_part1 = pd.read_csv(data_path + 'data_part1.csv', index_col = 0)
data_part2 = pd.read_csv(data_path + 'data_part2.csv', index_col = 0)

C:\Users\WENDY\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [71]:
print(f'The shape of data_part1 is {data_part1.shape}')
print(f'The shape of data_part2 is {data_part2.shape}')

data = pd.merge(data_part1, data_part2, on = ['PatientSerNum', 'date'], how = 'inner')
# data.sort_values(by = ['PatientSerNum', 'AppointmentSerNum', 'ScheduledStartTime', 'FractionNumber'], inplace = True)
print(f'The shape of data is {data.shape}')

The shape of data_part1 is (1047176, 21)
The shape of data_part2 is (507863, 14)
The shape of data is (1788434, 33)


In [78]:
feature_use = [
    'PatientSerNum', 'AppointmentSerNum',
    'dxt_AliasName', 'Sex', 'AliasSerNum',
    'ScheduledStartTime', 'ScheduledEndTime','ActualStartDate', 'ActualEndDate', 'Scheduled_duration', 'Actual_duration',
    'month', 'date', 'week','hour', 
    'DoctorSerNum', 'TreatmentOrientation', 
    'age', 
    'TreatmentStartTime', 'TreatmentEndTime',
    'FractionNumber', 'ImagesTaken', 'UserName',
    'RadiationId', 'MU', 'MUCoeff', 'CourseId'
]

In [82]:
print(data.shape)
data_ = data[feature_use]
data_.drop_duplicates(inplace = True)
data_.sort_values(by = ['PatientSerNum', 'AppointmentSerNum', 'ScheduledStartTime', 'FractionNumber'], inplace = True)
print(data_.shape)

(1788434, 33)


C:\Users\WENDY\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


(614793, 27)


C:\Users\WENDY\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [83]:
data_grouped = data_.groupby('PatientSerNum')
sample = data_grouped.get_group(6693)

In [84]:
sample

,PatientSerNum,AppointmentSerNum,dxt_AliasName,Sex,AliasSerNum,ScheduledStartTime,ScheduledEndTime,ActualStartDate,ActualEndDate,Scheduled_duration,...,age,TreatmentStartTime,TreatmentEndTime,FractionNumber,ImagesTaken,UserName,RadiationId,MU,MUCoeff,CourseId
18357,6693,571654,Breast,Female,23,2015-07-16 09:00:00,2015-07-16 09:15:00,2015-07-16 09:03:00,2015-07-16 09:11:00,15.0,...,71,2015-07-16 09:11:00,2015-07-16 09:11:00,2,0,ypoon,1.1 DIRECT EL_12,422.36,590.382,C2
18359,6693,571655,Breast,Female,23,2015-07-17 09:00:00,2015-07-17 09:15:00,2015-07-17 08:26:00,2015-07-17 08:46:00,15.0,...,71,2015-07-17 08:40:00,2015-07-17 08:40:00,3,0,ypoon,1.1 DIRECT EL_12,422.36,590.382,C2
18361,6693,571656,Breast,Female,23,2015-07-20 09:00:00,2015-07-20 09:15:00,2015-07-20 09:13:00,2015-07-20 09:40:00,15.0,...,71,2015-07-20 09:34:00,2015-07-20 09:34:00,4,0,snowac,1.1 DIRECT EL_12,422.36,590.382,C2
18363,6693,571657,Breast,Female,23,2015-07-21 09:15:00,2015-07-21 09:30:00,2015-07-21 08:51:00,2015-07-21 09:08:00,15.0,...,71,2015-07-21 09:07:00,2015-07-21 09:08:00,5,0,ypoon,1.1 DIRECT EL_12,422.36,590.382,C2
18365,6693,571658,Breast,Female,31,2015-07-15 09:00:00,2015-07-15 09:15:00,2015-07-15 08:50:00,2015-07-15 09:38:00,15.0,...,71,2015-07-15 09:37:00,2015-07-15 09:37:00,1,0,ypoon,1.1 DIRECT EL_12,422.36,590.382,C2


In [76]:
a = sample.loc[18357: 18358]
a

,PatientSerNum,AppointmentSerNum,dxt_AliasName,Sex,AliasSerNum,ScheduledStartTime,ScheduledEndTime,ActualStartDate,ActualEndDate,Scheduled_duration,...,age,TreatmentStartTime,TreatmentEndTime,FractionNumber,ImagesTaken,UserName,RadiationId,MU,MUCoeff,CourseId
18357,6693,571654,Breast,Female,23,2015-07-16 09:00:00,2015-07-16 09:15:00,2015-07-16 09:03:00,2015-07-16 09:11:00,15.0,...,71,2015-07-16 09:11:00,2015-07-16 09:11:00,2,0,ypoon,1.1 DIRECT EL_12,422.36,590.382,C2
18358,6693,571654,Breast,Female,23,2015-07-16 09:00:00,2015-07-16 09:15:00,2015-07-16 09:03:00,2015-07-16 09:11:00,15.0,...,71,2015-07-16 09:11:00,2015-07-16 09:11:00,2,0,ypoon,1.1 DIRECT EL_12,422.36,590.382,C2


In [77]:
for col in a.columns:
    if len(a[col].unique()) != 1:
        print(col)

PlanSerNum


In [75]:
sample

,PatientSerNum,AppointmentSerNum,dxt_AliasName,Sex,AliasSerNum,ScheduledStartTime,ScheduledEndTime,ActualStartDate,ActualEndDate,Scheduled_duration,...,age,TreatmentStartTime,TreatmentEndTime,FractionNumber,ImagesTaken,UserName,RadiationId,MU,MUCoeff,CourseId
18357,6693,571654,Breast,Female,23,2015-07-16 09:00:00,2015-07-16 09:15:00,2015-07-16 09:03:00,2015-07-16 09:11:00,15.0,...,71,2015-07-16 09:11:00,2015-07-16 09:11:00,2,0,ypoon,1.1 DIRECT EL_12,422.36,590.382,C2
18358,6693,571654,Breast,Female,23,2015-07-16 09:00:00,2015-07-16 09:15:00,2015-07-16 09:03:00,2015-07-16 09:11:00,15.0,...,71,2015-07-16 09:11:00,2015-07-16 09:11:00,2,0,ypoon,1.1 DIRECT EL_12,422.36,590.382,C2
18359,6693,571655,Breast,Female,23,2015-07-17 09:00:00,2015-07-17 09:15:00,2015-07-17 08:26:00,2015-07-17 08:46:00,15.0,...,71,2015-07-17 08:40:00,2015-07-17 08:40:00,3,0,ypoon,1.1 DIRECT EL_12,422.36,590.382,C2
18360,6693,571655,Breast,Female,23,2015-07-17 09:00:00,2015-07-17 09:15:00,2015-07-17 08:26:00,2015-07-17 08:46:00,15.0,...,71,2015-07-17 08:40:00,2015-07-17 08:40:00,3,0,ypoon,1.1 DIRECT EL_12,422.36,590.382,C2
18361,6693,571656,Breast,Female,23,2015-07-20 09:00:00,2015-07-20 09:15:00,2015-07-20 09:13:00,2015-07-20 09:40:00,15.0,...,71,2015-07-20 09:34:00,2015-07-20 09:34:00,4,0,snowac,1.1 DIRECT EL_12,422.36,590.382,C2
18362,6693,571656,Breast,Female,23,2015-07-20 09:00:00,2015-07-20 09:15:00,2015-07-20 09:13:00,2015-07-20 09:40:00,15.0,...,71,2015-07-20 09:34:00,2015-07-20 09:34:00,4,0,snowac,1.1 DIRECT EL_12,422.36,590.382,C2
18363,6693,571657,Breast,Female,23,2015-07-21 09:15:00,2015-07-21 09:30:00,2015-07-21 08:51:00,2015-07-21 09:08:00,15.0,...,71,2015-07-21 09:07:00,2015-07-21 09:08:00,5,0,ypoon,1.1 DIRECT EL_12,422.36,590.382,C2
18364,6693,571657,Breast,Female,23,2015-07-21 09:15:00,2015-07-21 09:30:00,2015-07-21 08:51:00,2015-07-21 09:08:00,15.0,...,71,2015-07-21 09:07:00,2015-07-21 09:08:00,5,0,ypoon,1.1 DIRECT EL_12,422.36,590.382,C2
18365,6693,571658,Breast,Female,31,2015-07-15 09:00:00,2015-07-15 09:15:00,2015-07-15 08:50:00,2015-07-15 09:38:00,15.0,...,71,2015-07-15 09:37:00,2015-07-15 09:37:00,1,0,ypoon,1.1 DIRECT EL_12,422.36,590.382,C2
18366,6693,571658,Breast,Female,31,2015-07-15 09:00:00,2015-07-15 09:15:00,2015-07-15 08:50:00,2015-07-15 09:38:00,15.0,...,71,2015-07-15 09:37:00,2015-07-15 09:37:00,1,0,ypoon,1.1 DIRECT EL_12,422.36,590.382,C2


In [60]:
sample.RadiationHstryAriaSer

18357    3278886
18358    3278886
18359    3279318
18360    3279318
18361    3279884
18362    3279884
18363    3280406
18364    3280406
18365    3278347
18366    3278347
Name: RadiationHstryAriaSer, dtype: int64

In [25]:
data_part2.columns

Index(['RadiationHstryAriaSer', 'TreatmentStartTime', 'TreatmentEndTime',
       'FractionNumber', 'ImagesTaken', 'UserName', 'RadiationSerNum',
       'RadiationId', 'ResourceSerNum', 'MU', 'MUCoeff', 'PatientSerNum',
       'CourseId', 'date'],
      dtype='object')

In [26]:
data_part1.columns

Index(['dxt_AliasName', 'DateOfBirth', 'Sex', 'AliasSerNum', 'PatientSerNum',
       'AppointmentSerNum', 'ScheduledStartTime', 'ScheduledEndTime',
       'ActualStartDate', 'ActualEndDate', 'CourseSerNum', 'DoctorSerNum',
       'PlanSerNum', 'TreatmentOrientation', 'age', 'month', 'date', 'week',
       'hour', 'Scheduled_duration', 'Actual_duration'],
      dtype='object')